In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, load_metric
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import json
import sys
parent_dir = Path(sys.path[0])
data_dir = parent_dir.parent.parent / 'data'
if str(data_dir) not in sys.path:
    sys.path.append(str(data_dir))
from timit.timit import get_timit
from nus.nus import get_nus
from jamendolyrics.jamendolyrics import get_jamendo
from CMUdict.utils import CMUDict, VOCAB
# Inspired by https://www.kaggle.com/code/vitouphy/phoneme-recognition-with-wav2vec2

/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DATA_DIR already in sys.path ['/mnt/storage/projects/programming/song-transcription/code/alignment', '/home/victor/miniconda3/envs/song-transcription/lib/python311.zip', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/lib-dynload', '', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg', '/tmp/tmps0c7juft', '/mnt/storage/projects/programming/song-transcription/data']


/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../../home/victor/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.1. Bad things might happen unless you revert torch to 1.x.
DATA_DIR already in sys.path ['/mnt/storage/projects/programming/song-transcription/code/alignment', '/home/victor/miniconda3/envs/song-transcription/lib/python311.zip', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/lib-dynload', '', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages', '/home/victor/miniconda3/envs/song-transcription/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg', '/tmp/tmps0c7juft', '/mnt/storage/projects/programming/song-transcription/data']


In [2]:
timit_dataset = get_timit()
timit_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'phonemes'],
        num_rows: 3696
    })
    test: Dataset({
        features: ['audio', 'text', 'phonemes'],
        num_rows: 1680
    })
    validation: Dataset({
        features: ['audio', 'text', 'phonemes'],
        num_rows: 924
    })
})

In [3]:
def change_item(item):
    item['phonetic'] = " ".join(item['phonemes'])
    return item
timit_dataset = timit_dataset.map(change_item)
# timit_dataset = timit_dataset.remove_columns(['phonemes'])
timit_row = timit_dataset['train'][0]
print(timit_row['text'])
# print(timit_row['phonemes'])
print(timit_row['phonetic'])
vocab = set(VOCAB.keys())
print(VOCAB)
print(len(vocab), vocab)
with open('working/vocab.json', 'w') as f:
    json.dump(VOCAB, f)

They assume no burglar will ever enter here.
SIL DH EY IH S UW M N OW SIL B ER SIL G L ER W L SIL EH V ER SIL EH N SIL T ER HH IH ER SIL
{'UH': 0, 'AW': 1, 'R': 2, 'SH': 3, 'OY': 4, 'AY': 5, 'UW': 6, 'P': 7, 'T': 8, 'B': 9, 'L': 10, 'S': 11, 'AH': 12, 'M': 13, 'D': 14, 'IH': 15, 'OW': 16, 'G': 17, 'IY': 18, 'DX': 19, 'K': 20, 'W': 21, 'SIL': 22, 'AE': 23, 'EH': 24, 'V': 25, 'EY': 26, 'NG': 27, 'CH': 28, 'ER': 29, 'N': 30, 'F': 31, '[UNK]': 32, 'Z': 33, 'HH': 34, 'JH': 35, 'DH': 36, 'Y': 37, '[PAD]': 38, 'AA': 39, 'TH': 40, '|': 41}
42 {'SH', 'UH', 'AW', 'R', 'OY', 'AY', 'UW', 'P', 'T', 'B', 'L', 'S', 'AH', 'M', 'D', '|', 'IH', 'OW', 'G', 'IY', 'DX', 'K', 'W', 'SIL', 'AE', 'EH', 'V', 'EY', 'NG', 'CH', 'ER', 'N', 'F', '[UNK]', 'Z', 'HH', 'JH', 'DH', 'Y', '[PAD]', 'AA', 'TH'}


In [4]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained('working/', unk_token='[UNK]', pad_token='[PAD]', word_delimiter_token='|')
tokenizer.vocab_size

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


42

In [ ]:
# tokenizer.tokenize(timit_row['phonetic'].split())

In [5]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16_000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [6]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='working/', vocab_size=42, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]', 'additional_special_tokens': [AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True)]}, clean_up_tokenization_spaces=True)

In [7]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    with processor.as_target_processor():
        batch["labels"] = processor(batch["phonetic"]).input_ids
    return batch
timit_dataset = timit_dataset.map(prepare_dataset)

Map: 100%|██████████| 924/924 [00:37<00:00, 24.69 examples/s]


In [8]:
import torch

from dataclasses import dataclass
from typing import Dict, List, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Copied from: https://www.kaggle.com/code/vitouphy/phoneme-recognition-with-wav2vec2s
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids)
    label_str = tokenizer.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {
        "wer": wer,
        "cer": cer
    }

In [13]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base-960h",
    attention_dropout=0.1,
    layerdrop=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.75,
    mask_time_length=10,
    mask_feature_prob=0.25,
    mask_feature_length=64,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True,
)
model.freeze_feature_extractor()
model

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder)

In [14]:
training_args = TrainingArguments(
    output_dir='working/base2',
    group_by_length=True,
    per_device_train_batch_size=8,
    # gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    gradient_checkpointing=True,
    fp16=True,
    max_steps=10000,
    save_steps=1000,  #100,
    eval_steps=1000,
    logging_steps=100,
    learning_rate=3e-5,
    warmup_steps=2000,
    save_total_limit=3,
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_dataset['train'],
    eval_dataset=timit_dataset['validation'],
    tokenizer=processor.feature_extractor
)
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 755574 KiB |   1487 MiB |   2033 GiB |   2032 GiB |
|       from large pool | 753920 KiB |   1486 MiB |   1981 GiB |   1980 GiB |
|       from small pool |   1654 KiB |      5 MiB |     52 GiB |     52 GiB |
|---------------------------------------------------------------------------|
| Active memory         | 755574 KiB |   1487 MiB |   2033 GiB |   2032 GiB |
|       from large pool | 753920 KiB |   1486 MiB |   1981 GiB |

In [15]:
trainer.train()

  1%|          | 100/10000 [01:40<1:53:26,  1.45it/s]

{'loss': 11.1329, 'learning_rate': 1.4850000000000002e-06, 'epoch': 0.22}


  2%|▏         | 200/10000 [03:18<1:54:12,  1.43it/s]

{'loss': 10.7518, 'learning_rate': 2.9700000000000004e-06, 'epoch': 0.43}


  3%|▎         | 300/10000 [04:59<1:48:17,  1.49it/s]

...

 88%|████████▊ | 8800/10000 [2:36:01<20:46,  1.04s/it]

{'loss': 1.0275, 'learning_rate': 4.52625e-06, 'epoch': 19.05}


 89%|████████▉ | 8900/10000 [2:37:41<19:01,  1.04s/it]

{'loss': 1.0284, 'learning_rate': 4.15125e-06, 'epoch': 19.26}


 89%|████████▉ | 8942/10000 [2:38:23<20:26,  1.16s/it]

In [ ]:
trainer.save_model()

In [9]:
model = Wav2Vec2ForCTC.from_pretrained(
    "working/base/checkpoint-7000-final",
    attention_dropout=0.1,
    layerdrop=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.75,
    mask_time_length=10,
    mask_feature_prob=0.25,
    mask_feature_length=64,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True,
)
model.freeze_feature_extractor()
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder)

In [10]:
training_args = TrainingArguments(
    output_dir='working/base/eval',
    group_by_length=True,
    per_device_train_batch_size=8,
    # gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    gradient_checkpointing=True,
    fp16=True,
    max_steps=10000,
    save_steps=1000,  #100,
    eval_steps=1000,
    logging_steps=100,
    learning_rate=3e-5,
    warmup_steps=2000,
    save_total_limit=3,
    load_best_model_at_end=True
)
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_dataset['train'],
    eval_dataset=timit_dataset['validation'],
    tokenizer=processor.feature_extractor
)
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 376050 KiB | 376050 KiB | 381442 KiB |   5392 KiB |
|       from large pool | 375040 KiB | 375040 KiB | 375040 KiB |      0 KiB |
|       from small pool |   1010 KiB |   5563 KiB |   6402 KiB |   5392 KiB |
|---------------------------------------------------------------------------|
| Active memory         | 376050 KiB | 376050 KiB | 381442 KiB |   5392 KiB |
|       from large pool | 375040 KiB | 375040 KiB | 375040 KiB |

In [11]:
trainer.evaluate(timit_dataset['test'])

100%|██████████| 210/210 [01:43<00:00,  2.02it/s]


{'eval_loss': 17.085832595825195,
 'eval_wer': 1.100810939736047,
 'eval_cer': 1.2895882947835449,
 'eval_runtime': 104.7644,
 'eval_samples_per_second': 16.036,
 'eval_steps_per_second': 2.004}

In [12]:
trainer.evaluate(timit_dataset['train'])

100%|██████████| 462/462 [03:46<00:00,  2.04it/s]


{'eval_loss': 16.69365119934082,
 'eval_wer': 1.103370954480903,
 'eval_cer': 1.2620423750326513,
 'eval_runtime': 227.2737,
 'eval_samples_per_second': 16.262,
 'eval_steps_per_second': 2.033}